In [130]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm


In [82]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 1:
        break
    cnt += 1

In [4]:
hits.shape

(134245, 7)

In [413]:
hits.shape

(100287, 7)

In [49]:
score

0.2365819488621897

In [16]:
score

0.23440822821310361

In [45]:
hits['z'].mean(0)

-34.57279

In [448]:
class F1Optimizer():
    def __init__(self):
        pass

    @staticmethod
    def get_expectations(P, pNone=None):
        expectations = []
        P = np.sort(P)[::-1]

        n = np.array(P).shape[0]
        DP_C = np.zeros((n + 2, n + 1))
        if pNone is None:
            pNone = (1.0 - P).prod()

        DP_C[0][0] = 1.0
        for j in range(1, n):
            DP_C[0][j] = (1.0 - P[j - 1]) * DP_C[0, j - 1]

        for i in range(1, n + 1):
            DP_C[i, i] = DP_C[i - 1, i - 1] * P[i - 1]
            for j in range(i + 1, n + 1):
                DP_C[i, j] = P[j - 1] * DP_C[i - 1, j - 1] + (1.0 - P[j - 1]) * DP_C[i, j - 1]

        DP_S = np.zeros((2 * n + 1,))
        DP_SNone = np.zeros((2 * n + 1,))
        for i in range(1, 2 * n + 1):
            DP_S[i] = 1. / (1. * i)
            DP_SNone[i] = 1. / (1. * i + 1)
        for k in range(n + 1)[::-1]:
            f1 = 0
            f1None = 0
            for k1 in range(n + 1):
                f1 += 2 * k1 * DP_C[k1][k] * DP_S[k + k1]
                f1None += 2 * k1 * DP_C[k1][k] * DP_SNone[k + k1]
            for i in range(1, 2 * k - 1):
                DP_S[i] = (1 - P[k - 1]) * DP_S[i] + P[k - 1] * DP_S[i + 1]
                DP_SNone[i] = (1 - P[k - 1]) * DP_SNone[i] + P[k - 1] * DP_SNone[i + 1]
            expectations.append([f1None + 2 * pNone / (2 + k), f1])

        return np.array(expectations[::-1]).T

    @staticmethod
    def maximize_expectation(P, pNone=None):
        expectations = F1Optimizer.get_expectations(P, pNone)

        ix_max = np.unravel_index(expectations.argmax(), expectations.shape)
        max_f1 = expectations[ix_max]

        predNone = True if ix_max[0] == 0 else False
        best_k = ix_max[1]

        return best_k, predNone, max_f1

    @staticmethod
    def _F1(tp, fp, fn):
        return 2 * tp / (2 * tp + fp + fn)

    @staticmethod
    def _Fbeta(tp, fp, fn, beta=1.0):
        beta_squared = beta ** 2
        return (1.0 + beta_squared) * tp / ((1.0 + beta_squared) * tp + fp + beta_squared * fn)

In [89]:
list(range(-1,2))

[-1, 0, 1]

In [154]:
def my_dist(x, y):
    return np.sqrt(((x - y)**2).sum())
    #return (x[0]-y[0])**2  + (x[1]-y[1])**2  + (x[2]-y[2])**2 

In [282]:
def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

In [250]:
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from sklearn.cluster.dbscan_ import dbscan
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np
import pandas as pd
import timeit
import multiprocessing
from multiprocessing import Pool

def find_labels(params):
    hits, dz = params
    a = hits['phi'].values
    z = hits['z'].values
    zr0 = hits['zr0'].values

    r = hits['r0'].values
    
    aa = a + np.sign(z) * dz * z
    #aa = a + dz * z

    f0 = np.cos(aa)
    f1 = np.sin(aa)
    f2 = hits['f2'].values
    f3 = zr0
    X = StandardScaler().fit_transform(np.column_stack([f0, f1, f3]))

    _, l = dbscan(X,  eps=0.0045, min_samples=1, n_jobs=8)
    return l + 1

def add_count(l):
    unique, reverse, count = np.unique(l, return_counts=True, return_inverse=True)
    c = count[reverse]
    c[np.where(l == 0)] = 0
    c[np.where(c > 20)] = 0
    return (l, c)

def do_dbscan_predict(hits, z_shift):
    start_time = timeit.default_timer()

    hits['z_shifted'] = hits['z'] - z_shift
    
    hits['r'] = np.sqrt(hits['x'] ** 2 + hits['y'] ** 2 + hits['z_shifted'] ** 2 )
    hits['r0'] = np.sqrt(hits['x'] ** 2 + hits['y'] ** 2)

    hits['zr0'] = hits['z_shifted'] / hits['r0']
    hits['phi'] = np.arctan2(hits['y'], hits['x'])
    hits['tmp'] = hits['r']/(2*hits['r0'])
    

    #scaler = MinMaxScaler(feature_range=(-1, 1))
    #hits['tmp'] = scaler.fit_transform(hits['tmp'].values.reshape(len(hits['tmp']), 1))
    #hits['tmp'] = hits['tmp'].apply(lambda x: min(0.99999, x))
    hits['f2'] = np.arctan(   hits['tmp']   )
    
    
    params = []
    for i in range(0, 60):
        dz = i * 0.00001
        params.append((hits, dz))
        if i > 0:
             params.append((hits, -dz))

             
    pool = Pool(processes=8)
    labels_for_all_steps = pool.map(find_labels, params)
    #results = [add_count(l) for l in labels_for_all_steps]
    results = labels_for_all_steps
    pool.close()

    labels = results[0]
    for i in range(1, len(results)):
        labels = merge(labels, results[i])

#     labels, counts = results[0]
#     for i in range(1, len(results)):
#         l, c = results[i]
#         idx = np.where((c - counts > 0))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts[idx] = c[idx]

    print('time spent:', timeit.default_timer() - start_time, z_shift)

    return labels, results

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

def run_dbscan():
    data_dir = '../input/train_1'

    sum = 0
    sum_score = 0

    z_result = []
    #its, cells, particles, truth = load_event(data_dir + '/event' + event_id)
    #for z_shift in range(-100, 101):
    for z_shift in range(1):
      
        labels, results = do_dbscan_predict(hits,  z_shift * 0.055)
        z_result += [labels]
        
    labels = z_result[0]
    for i in range(1, len(z_result)):
        labels = merge(labels, z_result[i])        

        
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    score = score_event(truth, submission)
    print('score ' , score)
    sum_score += score
    sum += 1

    print('--------------------------------------')
    print(sum_score / sum)
    
    return z_result, labels, submission

In [215]:
z_result = np.load('predicts/200_zshift.npy')
z_result = z_result[:, 0, :]

result_53 = np.load('predicts/53_all.npy')
result_53 = np.array(result_53)

In [219]:
result_53.shape

(600, 100287)

In [218]:
z_result.shape

(201, 100287)

In [228]:
result = np.vstack([z_result, result_53])

In [255]:
labels = z_result[0]
for i in range(1, len(z_result)):
    labels = merge(labels, z_result[i])

In [232]:
z_result

array([[    853824,     133123,     265120, ...,  921963395,  753213191,
        1321537499],
       [    853788,     133157,     265184, ...,    5988183,    1919331,
            503611],
       [    853680,     133118,     265138, ...,    5993086,    1919071,
            503544],
       ...,
       [    722639,     133198,     265088, ...,    6669604,     503583,
           1596370],
       [    722692,     133230,     265114, ...,    6668977,     503618,
           1596563],
       [    722791,     133252,     265196, ...,    6666979,     503715,
           1596720]])

In [240]:
tmp = np.array([add_count(l) for l in z_result])
cnts = tmp[:, 1, :]

In [236]:
z_result

array([[    853824,     133123,     265120, ...,  921963395,  753213191,
        1321537499],
       [    853788,     133157,     265184, ...,    5988183,    1919331,
            503611],
       [    853680,     133118,     265138, ...,    5993086,    1919071,
            503544],
       ...,
       [    722639,     133198,     265088, ...,    6669604,     503583,
           1596370],
       [    722692,     133230,     265114, ...,    6668977,     503618,
           1596563],
       [    722791,     133252,     265196, ...,    6666979,     503715,
           1596720]])

In [302]:
labels = range(z_result.shape[1])
labels = z_result[0]
for i in range(1, len(z_result)):
    labels = merge(labels, z_result[i], 0)

In [336]:
labels = range(z_result.shape[1])

for i in range(len(z_result)):
    labels = merge(labels, z_result[i], 0)

In [289]:
labels = ext_sub['track_id'].values

for i in range(len(z_result)):
    labels = merge(labels, z_result[i], 0)

In [362]:
#result = result_53
labels = range(result.shape[1])

for k in [18,16,14, 12, 10, 8, 6, 4, 0]:
#for k in [15,10,5, 0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    #submission = extend(submission,hits)
    #submission = extend(submission,hits)

    print(score_event(truth, submission))
    
    labels = submission['track_id'].values
   

0.02946572701478705
0.02946572701478705
0.05727761228580784
0.05727761228580784
0.11534065410312926
0.11534065410312926
0.23344574246875743
0.23344574246875743
0.36729231106049864
0.36729231106049864
0.45506391156244447
0.45506391156244447
0.4944932151899727
0.4944932151899727
0.5049299479307393
0.5049299479307393
0.5073244085906707
0.5073244085906707


In [361]:
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.5076282691673919


In [329]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5848926941183028


In [330]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5850779423213555


In [325]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5850817906028537


In [326]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5853420574657813


In [327]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5855217023763549


In [343]:
luis = np.load('predicts/luis_all.npy')

In [344]:
luis

array([[    0,     1,     2, ..., 74107, 74108, 74102],
       [    0,     1,     2, ..., 74048, 12232, 74042],
       [    0,     1,     2, ..., 73942, 73943, 73934],
       ...,
       [    0,     1,     2, ..., 86443, 86444, 86436],
       [    0,     1,     2, ..., 86518, 86519, 86508],
       [    0,     1,     2, ..., 86453, 86454, 86446]])

In [345]:
z_result

array([[    853824,     133123,     265120, ...,  921963395,  753213191,
        1321537499],
       [    853788,     133157,     265184, ...,    5988183,    1919331,
            503611],
       [    853680,     133118,     265138, ...,    5993086,    1919071,
            503544],
       ...,
       [    722639,     133198,     265088, ...,    6669604,     503583,
           1596370],
       [    722692,     133230,     265114, ...,    6668977,     503618,
           1596563],
       [    722791,     133252,     265196, ...,    6666979,     503715,
           1596720]])

In [347]:
labels = z_result[0]
for i in range(1, len(z_result)):
    labels = merge(labels, z_result[i], 0)

In [350]:
labels_luis = luis[0]
for i in range(1, len(z_result)):
    labels = merge(labels, luis[i], 0)

In [355]:
result = np.vstack([luis, z_result])

In [356]:
result.shape

(539, 100287)

In [360]:
result

array([[      0,       1,       2, ...,   75823,   75824,   75814],
       [      0,       1,       2, ...,   76479,   76480,   76471],
       [ 851709,  132852,  395171, ..., 6026819, 6215301,  502478],
       ...,
       [      0,       1,       2, ...,   83691,   83692,   83682],
       [      0,       1,       2, ...,   84777,   84778,   84769],
       [      0,       1,       2, ...,   75323,   75324,   75318]])

In [359]:
np.random.shuffle(result)

In [352]:
labels1 = merge(labels_luis, labels, 0)
submission = create_one_event_submission(0, hits['hit_id'].values, labels1)
print(score_event(truth, submission))

0.4913849820080143


In [ ]:
0.5076282691673919
0.5067205748968073
0.4731231005334312

In [ ]:
0.4912658962859528

In [ ]:
result = result_53
labels = range(z_result.shape[1])

for k in [15,10,5, 0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    submission = extend(submission,hits)
    print(score_event(truth, submission))
    
    labels = submission['track_id'].values
   

In [310]:
labels = range(z_result.shape[1])

for i in range(len(z_result)):
    labels = merge(labels, z_result[i], 15)
  
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
print(score_event(truth, submission))
submission = extend(submission,hits)
print(score_event(truth, submission))



labels = submission['track_id'].values

for i in range(len(z_result)):
    labels = merge(labels, z_result[i], 10)
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
print(score_event(truth, submission))
submission = extend(submission,hits)
print(score_event(truth, submission))



labels = submission['track_id'].values

for i in range(len(z_result)):
    labels = merge(labels, z_result[i], 5)
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
print(score_event(truth, submission))
submission = extend(submission,hits)
print(score_event(truth, submission))


0.07387727372951487
0.07835740005767651
0.34171814787412685
0.3529405051559642
0.4875550573063415
0.508872134715501


In [311]:
labels = submission['track_id'].values

for i in range(len(z_result)):
    labels = merge(labels, z_result[i], 0)
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
print(score_event(truth, submission))
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5042063878321094
0.516272597496374


In [290]:
labels

array([5133167003,     233409,     365406, ..., 5361331343, 5554215256,
       5920164732])

In [274]:
labels

array([2170322840,     233409,     365406, ...,     100284,     100285,
           100286])

In [264]:
labels.shape

(100287,)

In [303]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)

In [ ]:
results, labels, submission = run_dbscan()

In [307]:
score_event(truth, ext_sub)

0.5102744886009511

In [ ]:
0.3399329510827481
0.35145268798497153
0.4934600594722697
0.5137374103615819

0.516272597496374

In [ ]:
0.4844090174742257

In [ ]:
0.5079882204550195

In [ ]:
0.512355748910061

In [ ]:
0.5467987728575361

In [288]:
score_event(truth, ext_sub)

0.35145268798497153

In [ ]:
0.5855217023763549
0.5853420574657813
0.5850817906028537
0.5849290849426552
0.5827447401532195
0.5810983311646127
0.5807878755572605
0.5792614621063498
0.5102744886009511

In [306]:
ext_sub = extend(submission,hits)

 179.0000000

In [192]:
#from scipy.spatial import KDTree
from sklearn.neighbors import KDTree

In [309]:
	def extend(submission,hits):

		df = submission.merge(hits,  on=['hit_id'], how='left')
		df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
		df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
		df = df.assign(arctan2 = np.arctan2(df.z, df.r))

		for angle in range(-180,180,1):

		    #print ('\r %f'%angle, end='',flush=True)
		    #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
		    df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

		    min_num_neighbours = len(df1)
		    if min_num_neighbours<4: continue

		    hit_ids = df1.hit_id.values
		    x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
		    r  = (x**2 + y**2)**0.5
		    r  = r/1000
		    a  = np.arctan2(y,x)
		    tree = KDTree(np.column_stack([a,r]), metric='euclidean')

		    track_ids = list(df1.track_id.unique())
		    num_track_ids = len(track_ids)
		    min_length=3

		    for i in range(num_track_ids):
		        p = track_ids[i]
		        if p==0: continue

		        idx = np.where(df1.track_id==p)[0]
		        if len(idx)<min_length: continue

		        if angle>0:
		            idx = idx[np.argsort( z[idx])]
		        else:
		            idx = idx[np.argsort(-z[idx])]


		        ## start and end points  ##
		        idx0,idx1 = idx[0],idx[-1]
		        a0 = a[idx0]
		        a1 = a[idx1]
		        r0 = r[idx0]
		        r1 = r[idx1]

		        da0 = a[idx[1]] - a[idx[0]]  #direction
		        dr0 = r[idx[1]] - r[idx[0]]
		        direction0 = np.arctan2(dr0,da0) 

		        da1 = a[idx[-1]] - a[idx[-2]]
		        dr1 = r[idx[-1]] - r[idx[-2]]
		        direction1 = np.arctan2(dr1,da1) 

	 
		        ## extend start point
		        ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)
		        direction = np.arctan2(r0-r[ns],a0-a[ns])
		        ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]
	   
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

		        ## extend end point
		        ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)

		        direction = np.arctan2(r[ns]-r1,a[ns]-a1)
		        ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 
		        
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
		#print ('\r')
		df = df[['event_id', 'hit_id', 'track_id']]
		return df


In [175]:
results

[(array([ 850618,  132622,  264149, ...,  340948, 3557779,  501639]),
  array([ 7, 11, 12, ...,  3,  3,  7])),
 (array([    1,     2,     3, ..., 66312, 66313, 66306]),
  array([1, 2, 1, ..., 1, 1, 4])),
 (array([    1,     2,     3, ..., 66319, 66320, 22093]),
  array([ 1, 11,  1, ...,  1,  1,  5])),
 (array([    1,     2,     3, ..., 65932, 65933, 65925]),
  array([1, 2, 1, ..., 1, 1, 4])),
 (array([    1,     2,     3, ..., 65856, 65857, 65849]),
  array([ 1, 11, 12, ...,  1,  1,  4])),
 (array([    1,     2,     3, ..., 11416, 65417, 65412]),
  array([1, 2, 1, ..., 3, 1, 4])),
 (array([    1,     2,     3, ..., 65349, 65350, 65343]),
  array([ 1, 11, 12, ...,  1,  1,  4])),
 (array([    1,     2,     3, ..., 65074, 41881, 42029]),
  array([1, 2, 1, ..., 1, 2, 7])),
 (array([    1,     2,     3, ..., 65316, 65317, 65310]),
  array([ 1,  2, 12, ...,  1,  1,  4])),
 (array([    1,     2,     3, ..., 65332, 65333, 11520]),
  array([6, 2, 1, ..., 1, 1, 6])),
 (array([    1,     2,     3

In [176]:
results[0]

(array([ 850618,  132622,  264149, ...,  340948, 3557779,  501639]),
 array([ 7, 11, 12, ...,  3,  3,  7]))

In [ ]:
labels = results[0]
for i in range(1, len(results)):
    labels = merge(labels, results[i])

In [169]:
a = np.array([0, 1, 0])
b = np.array([1, 1, 0])
merge(a, b)

array([0, 2, 0])

In [144]:
100000 * 99999 / 2

4999950000.0

In [137]:
results = np.array(results)

In [138]:
np.save('predicts/200_zshift', results)

In [ ]:
0.4808449702457546
0.4293791663427201
0.33138997839109585

In [91]:
results, l = run_dbscan()

time spent: 18.721925161313266
time spent: 18.948315118905157
time spent: 18.731425021775067


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in arccos


time spent: 18.961979556828737
time spent: 19.04467044910416
time spent: 19.04395540105179
time spent: 19.009527281858027
time spent: 18.925030059646815
time spent: 19.136807607021183
time spent: 18.984390071127564
time spent: 19.243835468776524
time spent: 19.28255993919447
time spent: 19.141350229736418
time spent: 18.824109646957368
time spent: 18.964283721055835
time spent: 18.78620543796569
time spent: 18.925766750704497
time spent: 18.89491624990478
time spent: 19.07338073523715
time spent: 19.038074913900346
time spent: 19.202588236890733
time spent: 18.934166270308197
time spent: 18.601766844280064
time spent: 18.997709745075554
time spent: 18.646292409859598
time spent: 18.993201847653836
time spent: 19.019245713949203
time spent: 19.10756276687607
time spent: 18.537717562168837
time spent: 19.077655338216573
time spent: 18.94685385376215
time spent: 18.65020459331572
time spent: 18.550528069026768
time spent: 19.232760437764227
time spent: 19.17864443687722
time spent: 18.992

In [106]:
results

array([ 982775,  153102,  305064, ...,  392463, 7330348,  693000])

In [107]:
cl53 = np.load('predicts/53.npy')

In [108]:
cl53

array([20227238,  5345886,  7672948, ..., 35073462, 29223846,  3867059])

In [ ]:
labels, counts = results[0]

    l, c = results[i]
    idx = np.where((c - counts > 0))[0]
    labels[idx] = l[idx] + labels.max()
    counts[idx] = c[idx]    

In [122]:
m = merge(results, cl53)

In [127]:
submission = create_one_event_submission(0, hits['hit_id'].values, results)
score = score_event(truth, submission)

In [129]:
score

0.4293791663427201

In [93]:
results.shape

(100287,)

In [66]:
scaler.fit_transform(hits['tmp'].values.reshape(len(hits['tmp']), 1))

array([[0.29730186],
       [0.49345604],
       [0.3410686 ],
       ...,
       [0.04813972],
       [0.05025195],
       [0.0547769 ]], dtype=float32)

In [65]:
hits['tmp'].values.reshape(len(hits['tmp']), 1)

array([[ 7.37213  ],
       [11.906232 ],
       [ 8.383798 ],
       ...,
       [ 1.6127492],
       [ 1.6615734],
       [ 1.7661676]], dtype=float32)

In [34]:
l.mean()

1.0

In [ ]:
0.4293791663427201

In [417]:
np.array(results).shape

(119, 2, 100287)

In [492]:
results[:,0,:]

array([[ 850618,  132622,  264149, ...,  340948, 3557779,  501639],
       [      1,       2,       3, ...,   66312,   66313,   66306],
       [      1,       2,       3, ...,   66319,   66320,   22093],
       ...,
       [      1,       2,       3, ...,   70380,   70381,   70378],
       [      1,       2,       3, ...,   70427,   70428,   70420],
       [      1,       2,       3, ...,   46765,   70461,   70459]])

In [647]:
results = np.array(results)

In [449]:
optimizer = F1Optimizer()

In [569]:
scores = []

for i in range(1000):
    df = make_data(i)
    tmp = lgb.predict_proba(df.drop(['id', 'target'], axis=1))[:, 1]
    top_cnt = optimizer.maximize_expectation(tmp)[0]
    pred = np.in1d(tmp, -np.sort(-tmp)[:top_cnt])
    df['pred'] = pred


    scores += [accuracy_score( df['target'] , pred)]
scores = np.array(scores)
scores.mean()

/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [570]:
scores = scores[~np.isnan(scores)]

In [572]:
scores.mean()

0.8721194223521657

In [552]:
df

,cnt_edges,mean_len,max_len,std_len,comp_len,len_first,is_edge_first,id,target,pred
0,119,2.487395,15,2.358128,20,15,True,1001,True,True
1,4,0.504202,15,2.703480,20,15,True,1001,True,True
2,4,0.504202,15,2.703480,20,15,True,1001,True,True
3,4,0.504202,15,2.703480,20,15,True,1001,True,True
4,4,0.504202,15,2.703480,20,15,True,1001,True,True
5,4,0.504202,15,2.703480,20,15,True,1001,True,True
6,4,0.504202,15,2.703480,20,15,True,1001,True,True
7,4,0.504202,15,2.703480,20,15,True,1001,True,True
8,4,0.504202,15,2.703480,20,15,True,1001,True,True
9,1,0.033613,4,0.365135,20,0,False,1001,False,False


In [608]:
results.shape

(119, 2, 100287)

In [742]:
labels = np.zeros(results.shape[2])

In [640]:
labels1 = labels.copy()

In [695]:
import threading
lock = threading.Lock()



In [687]:
(labels!=0).sum()

1

In [749]:
def merge_foo(i):
    lock.acquire()

    if labels[i] == 0:
        lock.release()


        curr = np.equal(results[:, 0, :], np.repeat(results[:, 0, i:i+1], results.shape[2], axis=1))
        idxs = np.where( (curr.sum(axis=0) > 0) == True  )[0]
        cnt_edges = curr[:, idxs[1:]].sum(axis=0)#.sum()
        cnt = curr[:, idxs].sum(axis=1)
        lengths = curr[:, idxs[1:]] * np.repeat(cnt.reshape((len(cnt), 1)), len(idxs)-1, axis=1)

        df = pd.DataFrame()
        df['cnt_edges'] = cnt_edges
        df['mean_len'] = lengths.mean(axis=0)
        df['max_len'] = lengths.max(axis=0)
        df['std_len'] = lengths.std(axis=0)
        df['comp_len'] = len(idxs)

        # По несмещенному кластеру
        df['len_first'] = lengths[0]
        df['is_edge_first'] = curr[:, idxs[1:]][0]

        df['id'] = i

        #print(i)
        #lock.acquire()
        #print(i)

        tmp = lgb.predict_proba(df.drop(['id'], axis=1))[:, 1]
        #lock.release()
        #print(i)

        #print(i)


        #lock.acquire()
        
        top_cnt = optimizer.maximize_expectation(tmp)[0]
        #lock.release()
        #print(i)
        
        pred = np.in1d(tmp, -np.sort(-tmp)[:top_cnt])
        df['pred'] = pred    

    #     particle_id = truth[truth.hit_id == idxs[0]+1]['particle_id'].values[0]
    #     gt_hit_ids = truth[truth.particle_id == particle_id]['hit_id'].values
    #     hit_ids = idxs + 1
    #     df['target'] = np.in1d(hit_ids[1:], gt_hit_ids)

        true_idxs_mask = np.array([True] + list(df['pred'].values))
        true_idxs = idxs[true_idxs_mask]

        lock.acquire()
        if labels[idxs[0]] == 0:
            labels[true_idxs] = i+1
        lock.release()
        #print(i)

    else:
        lock.release()



In [710]:
merge_foo(0)

0


In [711]:
labels

array([1., 1., 2., ..., 0., 0., 0.])

In [750]:
%%time
pool = Pool(processes=8)

import threading
lock = threading.Lock()


#pool.map(merge_foo,  range(results.shape[2]))

for df in tqdm(pool.imap(merge_foo,  range(results.shape[2]))):
    None
    
pool.close()


0it [00:00, ?it/s]
1it [00:01,  1.05s/it]
3it [00:01,  2.61it/s]
9it [00:02,  4.45it/s]
12it [00:02,  5.62it/s]
17it [00:02,  5.69it/s]
19it [00:03,  5.99it/s]
25it [00:03,  6.38it/s]
26it [00:04,  6.30it/s]
34it [00:04,  7.03it/s]
35it [00:05,  6.86it/s]
38it [00:42,  1.11s/it]
364it [00:42,  8.58it/s]
392it [00:45,  8.63it/s]
412it [00:47,  8.68it/s]
426it [00:49,  8.68it/s]
437it [00:49,  8.74it/s]
445it [00:50,  8.76it/s]
451it [00:51,  8.81it/s]
456it [00:51,  8.82it/s]
460it [00:52,  8.83it/s]
463it [00:52,  8.82it/s]
466it [00:52,  8.86it/s]
469it [00:53,  8.83it/s]
471it [00:53,  8.84it/s]
475it [00:53,  8.87it/s]
477it [00:53,  8.85it/s]
479it [00:54,  8.85it/s]
483it [00:54,  8.88it/s]
485it [00:54,  8.88it/s]
487it [00:54,  8.88it/s]
490it [00:55,  8.90it/s]
492it [00:55,  8.91it/s]
494it [00:55,  8.88it/s]
498it [00:55,  8.92it/s]
500it [00:56,  8.93it/s]
502it [00:56,  8.93it/s]
504it [00:56,  8.92it/s]
506it [00:56,  8.92it/s]
508it [00:56,  8.94it/s]
510it [00:56,  8.95

2647it [02:33, 17.26it/s]
2651it [02:33, 17.26it/s]
2659it [02:33, 17.30it/s]
2664it [02:33, 17.32it/s]
2669it [02:34, 17.33it/s]
2674it [02:34, 17.35it/s]
2678it [02:34, 17.35it/s]
2686it [02:34, 17.38it/s]
2694it [02:34, 17.41it/s]
2699it [02:34, 17.43it/s]
2705it [02:34, 17.46it/s]
2710it [02:35, 17.45it/s]
2715it [02:35, 17.48it/s]
2719it [02:35, 17.48it/s]
2726it [02:35, 17.50it/s]
2733it [02:35, 17.53it/s]
2737it [02:36, 17.54it/s]
2744it [02:36, 17.57it/s]
2749it [02:36, 17.59it/s]
2754it [02:36, 17.60it/s]
2759it [02:36, 17.62it/s]
2764it [02:36, 17.63it/s]
2768it [02:36, 17.64it/s]
2774it [02:37, 17.67it/s]
2779it [02:37, 17.67it/s]
2788it [02:37, 17.71it/s]
2793it [02:37, 17.72it/s]
2802it [02:37, 17.76it/s]
2807it [02:37, 17.78it/s]
2812it [02:38, 17.79it/s]
2820it [02:38, 17.82it/s]
2825it [02:38, 17.83it/s]
2834it [02:38, 17.87it/s]
2839it [02:38, 17.89it/s]
2845it [02:38, 17.90it/s]
2852it [02:39, 17.93it/s]
2857it [02:39, 17.94it/s]
2864it [02:39, 17.97it/s]
2870it [02:3

KeyboardInterrupt: 

Process ForkPoolWorker-631:
Process ForkPoolWorker-632:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-633:
Traceback (most recent call last):
Process ForkPoolWorker-636:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "<ipython-input-749-d4d842a8a6ad>", line 8, in merge_foo
    curr = np.equal(results[:, 0, :], np.repeat(r

In [617]:
#for i in tqdm(np.random.choice(range(results.shape[2]), 1000)):
label_id = 1
for i in tqdm(range(results.shape[2])):
    
    lock.acquire()
    if labels[i] != 0:
        continue
    lock.release()
    
    curr = np.equal(results[:, 0, :], np.repeat(results[:, 0, i:i+1], results.shape[2], axis=1))
    idxs = np.where( (curr.sum(axis=0) > 0) == True  )[0]
    cnt_edges = curr[:, idxs[1:]].sum(axis=0)#.sum()
    cnt = curr[:, idxs].sum(axis=1)
    lengths = curr[:, idxs[1:]] * np.repeat(cnt.reshape((len(cnt), 1)), len(idxs)-1, axis=1)

    df = pd.DataFrame()
    df['cnt_edges'] = cnt_edges
    df['mean_len'] = lengths.mean(axis=0)
    df['max_len'] = lengths.max(axis=0)
    df['std_len'] = lengths.std(axis=0)
    df['comp_len'] = len(idxs)

    # По несмещенному кластеру
    df['len_first'] = lengths[0]
    df['is_edge_first'] = curr[:, idxs[1:]][0]

    df['id'] = i

    
    tmp = lgb.predict_proba(df.drop(['id'], axis=1))[:, 1]
    top_cnt = optimizer.maximize_expectation(tmp)[0]
    pred = np.in1d(tmp, -np.sort(-tmp)[:top_cnt])
    df['pred'] = pred    
    
#     particle_id = truth[truth.hit_id == idxs[0]+1]['particle_id'].values[0]
#     gt_hit_ids = truth[truth.particle_id == particle_id]['hit_id'].values
#     hit_ids = idxs + 1
#     df['target'] = np.in1d(hit_ids[1:], gt_hit_ids)

    true_idxs_mask = np.array([True] + list(df['pred'].values))
    true_idxs = idxs[true_idxs_mask]
    
    lock.acquire()
    if labels[idxs[0]] != 0:
        labels[true_idxs] = label_id
    lock.release()

    
    label_id += 1


  0%|          | 158/100287 [00:08<1:28:49, 18.79it/s]
Exception in thread Thread-92:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 100287/100287 [51:13<00:00, 32.63it/s]


In [631]:
np.set_printoptions(precision=3, threshold=100)

labels.astype('int')

array([ 1765,     2,  2535, ...,     0,     0, 36456])

In [634]:
l

array([ 850618,  132622,  264149, ...,  340948, 3557779,  501639])

In [638]:
np.unique(l).shape

(37240,)

In [639]:
np.unique(labels).shape

(20843,)

In [620]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
score = score_event(truth, submission)

In [621]:
score

0.21629635754461213

In [635]:
(labels != 0).sum()

80002

In [606]:
true_idxs

array([32, 61])

In [605]:
df

,cnt_edges,mean_len,max_len,std_len,comp_len,len_first,is_edge_first,id,pred
0,119,2.218487,6,0.790273,15,6,True,999,True
1,5,0.176471,6,0.894869,15,6,True,999,False
2,1,0.042017,5,0.456419,15,0,False,999,False
3,2,0.100840,6,0.771280,15,6,True,999,False
4,2,0.100840,6,0.771280,15,6,True,999,False
5,1,0.042017,5,0.456419,15,0,False,999,False
6,2,0.100840,6,0.771280,15,6,True,999,False
7,1,0.042017,5,0.456419,15,0,False,999,False
8,2,0.084034,5,0.642734,15,0,False,999,False
9,2,0.084034,5,0.642734,15,0,False,999,False


In [595]:
results

array([[[ 850618,  132622,  264149, ...,  340948, 3557779,  501639],
        [      7,      11,      12, ...,       3,       3,       7]],

       [[      1,       2,       3, ...,   66312,   66313,   66306],
        [      1,       2,       1, ...,       1,       1,       4]],

       [[      1,       2,       3, ...,   66319,   66320,   22093],
        [      1,      11,       1, ...,       1,       1,       5]],

       ...,

       [[      1,       2,       3, ...,   70380,   70381,   70378],
        [      1,       2,       1, ...,       1,       1,       2]],

       [[      1,       2,       3, ...,   70427,   70428,   70420],
        [      1,       2,       1, ...,       1,       1,       3]],

       [[      1,       2,       3, ...,   46765,   70461,   70459],
        [      1,       2,       1, ...,       2,       1,       2]]])

In [579]:
idxs

array([  32,   61, 1266, 2676, 2700, 2729, 2741, 2789, 4441, 4485, 4523,
       6503, 6514, 6596, 8912])

In [580]:
idxs.shape

(15,)

In [578]:
df

,cnt_edges,mean_len,max_len,std_len,comp_len,len_first,is_edge_first,id,target
0,119,2.218487,6,0.790273,15,6,True,999,True
1,5,0.176471,6,0.894869,15,6,True,999,True
2,1,0.042017,5,0.456419,15,0,False,999,False
3,2,0.100840,6,0.771280,15,6,True,999,True
4,2,0.100840,6,0.771280,15,6,True,999,True
5,1,0.042017,5,0.456419,15,0,False,999,False
6,2,0.100840,6,0.771280,15,6,True,999,True
7,1,0.042017,5,0.456419,15,0,False,999,False
8,2,0.084034,5,0.642734,15,0,False,999,False
9,2,0.084034,5,0.642734,15,0,False,999,False


In [581]:
df.shape

(14, 9)

In [395]:
%%time


def make_data(i):
    #X = pd.DataFrame()

    #for i in tqdm(np.random.choice(range(results.shape[2]), 1000)):
    curr = np.equal(results[:, 0, :], np.repeat(results[:, 0, i:i+1], results.shape[2], axis=1))
    idxs = np.where( (curr.sum(axis=0) > 0) == True  )[0]
    cnt_edges = curr[:, idxs[1:]].sum(axis=0)#.sum()
    cnt = curr[:, idxs].sum(axis=1)
    lengths = curr[:, idxs[1:]] * np.repeat(cnt.reshape((len(cnt), 1)), len(idxs)-1, axis=1)

    df = pd.DataFrame()
    df['cnt_edges'] = cnt_edges
    df['mean_len'] = lengths.mean(axis=0)
    df['max_len'] = lengths.max(axis=0)
    df['std_len'] = lengths.std(axis=0)
    df['comp_len'] = len(idxs)

    # По несмещенному кластеру
    df['len_first'] = lengths[0]
    df['is_edge_first'] = curr[:, idxs[1:]][0]

    df['id'] = i

    particle_id = truth[truth.hit_id == idxs[0]+1]['particle_id'].values[0]
    gt_hit_ids = truth[truth.particle_id == particle_id]['hit_id'].values
    hit_ids = idxs + 1
    df['target'] = np.in1d(hit_ids[1:], gt_hit_ids)

    #X = pd.concat([X, df], axis=0)

    #X.index = range(len(X))
    return df

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [546]:
%%time
pool = Pool(processes=8)


X = pd.DataFrame()

for df in tqdm(pool.imap(make_data,  np.random.choice(range(results.shape[2]), 1000))):
    X = pd.concat([X, df], axis=0)
     
pool.close()
X.index=range(len(X))

1000it [00:20, 49.08it/s]

CPU times: user 7.39 s, sys: 224 ms, total: 7.62 s
Wall time: 20.5 s


In [348]:
np.random.choice(range(10), 11)

array([7, 1, 3, 8, 5, 0, 0, 8, 4, 5, 9])

In [547]:
from sklearn.model_selection import GroupKFold

kfold = GroupKFold(n_splits=5)

folds = []
fold_id = 0
X['fold'] = np.nan
for itr, ite in kfold.split(X, X['target'], X['id']):
    X.ix[ite, 'fold'] = fold_id
    folds += [[itr, ite]]
    fold_id += 1


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [549]:
1 - X['target'].mean()

0.8997144896502498

In [733]:
#%%time
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss


scores = []
for fold_id in range(5):
    lgb = lgbm.sklearn.LGBMClassifier(num_threads=1)
    lgb.fit(X[X.fold != fold_id].drop(['target', 'fold', 'id'],axis=1), X[X.fold != fold_id]['target'] )

    pred = lgb.predict(X[X.fold == fold_id].drop(['target', 'fold', 'id'],axis=1))#[:,1]
    
    proba = lgb.predict_proba(X[X.fold == fold_id].drop(['target', 'fold', 'id'],axis=1))[:, 1]
    #X.ix[X[X.fold == fold_id].index, 'pred'] = proba    
#     top_cnt = optimizer.maximize_expectation(proba)[0]
#     pred = np.in1d(proba, -np.sort(-proba)[:top_cnt])
    
#     scores += [accuracy_score( X[X.fold == fold_id]['target'] , pred)]
    scores += [log_loss(X[X.fold == fold_id]['target'] , proba)]
scores = np.array(scores)
print(scores.mean(), scores.std())



# pred = lgb.predict(X_val.drop(['target'],axis=1))#[:,1]
# print(accuracy_score( X_val['target'], pred))

# pred = lgb.predict_proba(X_val.drop(['target'],axis=1))[:,1]
# print(roc_auc_score( X_val['target'], pred))

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.22918594238925846 0.024179816568958106


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [535]:
def get_pred(proba):
    #proba = proba.tolist()
    top_cnt = optimizer.maximize_expectation(proba)[0]
    pred = np.in1d(proba, -np.sort(-proba)[:top_cnt])
    return pred

In [519]:
X.ix[X[X.fold == fold_id].index, 'pred'] = proba

In [521]:
X['pred'].unique()

array([       nan, 0.9368907 , 0.77258251, ..., 0.02098584, 0.02402272,
       0.01993065])

In [410]:
lgb.feature_importances_

array([ 205,  373,  466,  536, 1071,  349])

In [ ]:
0.9460906351722684 0.008051259262934596

In [539]:
tmp = X.groupby('id')['pred'].apply(lambda x: get_pred(np.array(x.tolist())))

In [543]:
tmp

id
27        [True, True, True, True, True, True, True, Tru...
64        [True, False, False, True, True, True, True, T...
386       [True, True, True, True, True, True, True, Tru...
393                                           [True, False]
583       [True, False, True, False, False, False, False...
604       [True, True, True, True, True, True, True, Fal...
950       [True, True, True, True, True, True, True, Tru...
1023      [True, True, True, False, True, False, False, ...
1175      [False, False, False, False, False, False, Fal...
1235      [True, True, True, True, True, True, True, Tru...
1371      [False, True, True, True, True, False, True, F...
1553      [True, True, True, False, True, True, False, T...
1603      [True, True, True, True, True, True, True, Tru...
1792      [True, True, True, True, True, True, True, Tru...
1895      [True, True, True, False, True, True, True, Tr...
1897                                                [False]
1962      [True, False, False, True, 

In [295]:
lengths.shape

(119, 11)

In [304]:
hit_ids in gt_hit_ids

True

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [303]:
hit_ids

array([    2,  1791,  1857,  3955,  4028,  6496,  6553,  6619,  9501,
        9618,  9693, 12998])

In [314]:
tmp = list(hit_ids) + [1000]

In [260]:
hits.shape

(134245, 10)

In [278]:
idxs

array([ 0, 53])

In [293]:
gt_hit_ids

array([    2,  1791,  1857,  3955,  4028,  6496,  6553,  6619,  9501,
        9618,  9693, 12998], dtype=int32)

In [286]:
idxs + 1

array([    2,  1791,  1857,  3955,  4028,  6496,  6553,  6619,  9501,
        9618,  9693, 12998])

In [250]:
X

,cnt_edges,mean_len,max_len,std_len
0,4,0.378151,0.378151,0.378151
1,4,0.378151,0.378151,0.378151
2,4,0.378151,0.378151,0.378151
3,4,0.378151,0.378151,0.378151
4,4,0.378151,0.378151,0.378151
5,4,0.378151,0.378151,0.378151
6,4,0.378151,0.378151,0.378151
7,4,0.378151,0.378151,0.378151
8,3,0.302521,0.302521,0.302521
9,3,0.302521,0.302521,0.302521


In [220]:
len(cnt)

119

In [159]:
import itertools

In [161]:
for i in itertools.combinations(idxs, 2):
    print(i)

(2, 9438)
(2, 9538)
(2, 12914)
(2, 12985)
(2, 13061)
(2, 16530)
(2, 16631)
(9438, 9538)
(9438, 12914)
(9438, 12985)
(9438, 13061)
(9438, 16530)
(9438, 16631)
(9538, 12914)
(9538, 12985)
(9538, 13061)
(9538, 16530)
(9538, 16631)
(12914, 12985)
(12914, 13061)
(12914, 16530)
(12914, 16631)
(12985, 13061)
(12985, 16530)
(12985, 16631)
(13061, 16530)
(13061, 16631)
(16530, 16631)


In [ ]:
%%time
np.equal(results[:, 0, :], np.repeat(results[:, 0, 2:3], results.shape[2], axis=1))

In [148]:
results[:, 0, 2:3]

array([[4136909],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [      3],
       [  

In [101]:
(results[0, 0, :] == 1).sum()

2

In [103]:
results[:, 1, :]

array([[ 2, 12,  4, ...,  4,  4,  5],
       [ 2,  1,  1, ...,  1,  1,  1],
       [ 2,  1,  1, ...,  1,  1,  1],
       ...,
       [ 2,  1,  1, ...,  1,  2,  1],
       [ 2,  1,  1, ...,  1,  1,  1],
       [ 2,  1,  1, ...,  1,  1,  1]])

In [95]:
np.unique(results[20:21, 0, :]).shape

(82700,)

In [ ]:
0.40451986708955157
0.4036879581113626
0.40262585370782106

In [ ]:
0.37713172974254583

In [37]:
unique, reverse, count = np.unique([0, 0, 0, 1, 1, 2, 7, 7, 7], return_counts=True, return_inverse=True)


In [38]:
unique

array([0, 1, 2, 7])

In [39]:
count

array([3, 2, 1, 3])

In [40]:
reverse

array([0, 0, 0, 1, 1, 2, 3, 3, 3])

In [41]:
count[reverse]

array([3, 3, 3, 2, 2, 1, 3, 3, 3])

In [ ]:
[0, 0, 0, 1, 1, 2, 7, 7, 7]
[3, 3, 3, 2, 2, 1, 3, 3, 3]

In [ ]:
[0, 0, 0, 0, 1, 2, 7, 7, 7]
[4, 4, 4, 4, 2, 1, 3, 3, 3]

In [ ]:
labels, counts = results[0]
for i in range(1, len(results)):
    l, c = results[i]
    idx = np.where((c - counts > 0))[0]
    labels[idx] = l[idx] + labels.max()
    counts[idx] = c[idx]

In [45]:
np.where((count[reverse] - reverse > 0))

(array([0, 1, 2, 3, 4]),)

In [44]:
reverse - count[reverse] 

array([-3, -3, -3, -1, -1,  1,  0,  0,  0])

In [46]:
count[reverse] - reverse 

array([ 3,  3,  3,  1,  1, -1,  0,  0,  0])